The code here originated with a Blog post by Dan Aloni. Here is the link:

https://blog.aloni.org/posts/backprop-with-tensorflow/

Please do visit this blog since the commentary included there is valuable and despite embedding this example in Jupyter is not full carried across to here. 

Some context - I have greatly appreciated Dan Alone's original post and we have developed several variants over the past three years in CS510. Important for this version is it has been updated to run with TF2.

New here relative to the original blog post is code to visualize what the first hidden layer is learning.  This hopefully provides additional insight into the feature learning process - something which for more complex networks is extremely hard to intuit. 

Running on a MacBook Pro with Catalina the following will buid the necessary environment from scratch using anaconda. 

`conda create --name aloni2 ...................`

Ross Beveridge
April 8, 2020

In [1]:
# Most of this is standard.  The two lines using the os package are a hack I found necessary to run on Catalina.
# The adaptation to use a local version of the dataset library makes for a more self contained example. 

import matplotlib
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'
matplotlib.use('Qt5Agg')

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import tensorflow as tf

This version has gone back to the original data loading and handling in the original blog post.  For awhile we used an alternative.  Key to making this work is sometimes needing to hand load the support code for the MNIST data. Here are detailed instructions:

There will be a folder in your storage space for 'anaconda' which contains all the files for anaconda distribution (fore me it is 'anaconda3').

Inside this folder you will find 'envs' folder, which has sub-folders for anaconda environments.

One of these will be the tensorflow 2 environment folder, the folder name will be the same as your environment name. The tensorflow package is installed in the python site-packages. Inside your environment folder you will find the following folder path:

'lib/python3.x/site-packages/tensorflow_core/`

Place the files from https://github.com/tensorflow/tensorflow/tree/master/tensorflow/examples/tutorials/mnist in the subfolder examples/tutorials/mnist . On my installation the full path looks like ‘’.

`/Users/ross/opt/anaconda3/envs/tfcv/lib/python3.7/site-packages/tensorflow_core/examples/tutorials/mnist`

My thanks to Saurabh Deotale for setting me on the correct path to make this option work. 



In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as: tensorflow_datasets.load('mnist')
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/_DataSet.py from tensorflow/models.


Below we establish the variables which constitute the layers of the network, the inputs a_0, the outputs y, and the internal weights and offsets.  Note the randomized initialization of the weights and offsets is on by default. Go ahead, turn it off and start with nice clean zeroes for all the weights and offsets. 

In [3]:
tf.compat.v1.disable_eager_execution()

a_0 = tf.compat.v1.placeholder(tf.float32, [None, 784])
y = tf.compat.v1.placeholder(tf.float32, [None, 10])

middle  = 30
raninit = True

if raninit :
    w_1 = tf.Variable(tf.random.truncated_normal([784, middle]))
    b_1 = tf.Variable(tf.random.truncated_normal([1, middle]))
    w_2 = tf.Variable(tf.random.truncated_normal([middle, 10]))
    b_2 = tf.Variable(tf.random.truncated_normal([1, 10]))
else :
    w_1 = tf.Variable(tf.zeros([784, middle]))
    b_1 = tf.Variable(tf.zeros([1, middle]))
    w_2 = tf.Variable(tf.zeros([middle, 10]))
    b_2 = tf.Variable(tf.zeros([1, 10]))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


This next code block makes the non-linear function mapping input to output for units and that function's first partial derivative explicit.  

In [4]:
def sigma(x):
    return tf.compat.v1.div(tf.constant(1.0),
                  tf.add(tf.constant(1.0), tf.exp(tf.negative(x))))

z_1 = tf.add(tf.matmul(a_0, w_1), b_1)
a_1 = sigma(z_1)
z_2 = tf.add(tf.matmul(a_1, w_2), b_2)
a_2 = sigma(z_2)

def sigmaprime(x):
    return tf.multiply(sigma(x), tf.subtract(tf.constant(1.0), sigma(x)))

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


The next body of code establishes what is in effect a second network which carries out the backpropogation calculation. Note in particular the error signal `diff` and the learning rate `eta`.  

In [5]:
diff = tf.subtract(a_2, y)

d_z_2 = tf.multiply(diff, sigmaprime(z_2))
d_b_2 = d_z_2
d_w_2 = tf.matmul(tf.transpose(a=a_1), d_z_2)

d_a_1 = tf.matmul(d_z_2, tf.transpose(a=w_2))
d_z_1 = tf.multiply(d_a_1, sigmaprime(z_1))
d_b_1 = d_z_1
d_w_1 = tf.matmul(tf.transpose(a=a_0), d_z_1)

eta = tf.constant(0.5)
step = [
    tf.compat.v1.assign(w_1,
            tf.subtract(w_1, tf.multiply(eta, d_w_1)))
  , tf.compat.v1.assign(b_1,
            tf.subtract(b_1, tf.multiply(eta,
                               tf.reduce_mean(input_tensor=d_b_1, axis=[0]))))
  , tf.compat.v1.assign(w_2,
            tf.subtract(w_2, tf.multiply(eta, d_w_2)))
  , tf.compat.v1.assign(b_2,
            tf.subtract(b_2, tf.multiply(eta,
                               tf.reduce_mean(input_tensor=d_b_2, axis=[0]))))]

The actual code to train and report on the performance of the network follows.  Here the code departs from the origianl Blog post in several ways including the introduction of the ability to visually inspect hidden layers.

In [6]:
acct_mat = tf.equal(tf.argmax(input=a_2, axis=1), tf.argmax(input=y, axis=1))
acct_res = tf.reduce_sum(input_tensor=tf.cast(acct_mat, tf.float32))

sess = tf.compat.v1.InteractiveSession()
w_1_before = ''
w_1_after  = ''

def run_training(n,m) :
    global w_1_before
    global w_1_after
    k = 0
    sess.run(tf.compat.v1.global_variables_initializer())
    w_1_before = sess.run(w_1)
    for i in range(n):
        batch_xs, batch_ys = mnist.train.next_batch(10)
        sess.run(step, feed_dict = {a_0: batch_xs,
                                y : batch_ys})
        if i % m == 0:
            k = k + 1
            res = sess.run(acct_res, feed_dict =
                       {a_0: mnist.test.images[:1000],
                        y : mnist.test.labels[:1000]})
            stout = 'Step ' + repr(k) + ' Num Correct ' + repr(res)
            print(stout)
            #print(sess.run(w_1)[404:409,0])
    w_1_after = sess.run(w_1)


In [7]:
run_training(10000,1000)

Step 1 Num Correct 95.0
Step 2 Num Correct 772.0
Step 3 Num Correct 816.0
Step 4 Num Correct 817.0
Step 5 Num Correct 829.0
Step 6 Num Correct 828.0
Step 7 Num Correct 825.0
Step 8 Num Correct 839.0
Step 9 Num Correct 834.0
Step 10 Num Correct 846.0


The next three functions are useful for actually exploring the network weights before and after training

In [ ]:
def show_weight_before(node) :
    dimg = np.reshape(w_1_before[:,node],(28,28))
    plt.imshow(dimg)
    plt.show()
    
def show_weight_after(node) :
    dimg = np.reshape(w_1_after[:,node],(28,28))
    plt.imshow(dimg)
    plt.show()
    
def show_weight_change(node) :
    diff = w_1_before - w_1_after
    dimg = np.reshape(diff[:,node],(28,28))
    plt.imshow(dimg)
    plt.show()